## 🕺🕺🕺 Install Environment

## Import all the necessary repositories from github as well as installing the libraries and packages

In [1]:
print('Git clone project and install requirements...')
!rm -rf ict3104-team01-2023 && git clone https://github.com/FS75/ict3104-team01-2023.git
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

Git clone project and install requirements...
Cloning into 'ict3104-team01-2023'...
remote: Enumerating objects: 1401, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 1401 (delta 51), reused 151 (delta 35), pack-reused 1205
Receiving objects: 100% (1401/1401), 3.41 GiB | 38.59 MiB/s, done.
Resolving deltas: 100% (174/174), done.
Updating files: 100% (1015/1015), done.
Cloning into 'mmpose'...
remote: Enumerating objects: 30322, done.
remote: Counting objects: 100% (1098/1098), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 30322 (delta 571), reused 947 (delta 525), pack-reused 29224
Receiving objects: 100% (30322/30322), 52.76 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (21370/21370), done.


In [2]:
!cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!export PYTHONPATH=/content/ict3104-team01-2023/FollowYourPoseTeam1:$PYTHONPATH
!python3.10 -m pip install -q -U --pre triton
!apt update
!python3.10 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,473 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,278 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/

In [3]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!python3.10 -m pip install -r requirements.txt
!pip install tensorrt

/content/ict3104-team01-2023/FollowYourPoseTeam1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

## 💃💃💃 Weight

In [4]:
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/ict3104-team01-2023/FollowYourPoseTeam1/FollowYourPose_v1/* /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/
%rm -rf FollowYourPose_v1

Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 3.95 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 51.94 MiB/s, done.


##**Data Exploration Section** 🔍🔍🔍
####This section loads, lists and displays video data from the Charades project. Here you can view the video playback of your chosen file.
-------------------
  
####How to use:
1. Click "Run Cell", the play button on the left
2. Select a video from the dropdown menu.
3. Click "Play Video"
4. Rerun the cell if the videos in the select video drop box does not refresh

In [15]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from moviepy.editor import VideoFileClip

# Function to update the file selector based on folder selection
def update_file_selector(*args):
    folder = folder_selector.value
    subfolder_path = os.path.join(base_path, folder)
    # List all mp4 files in the selected subfolder
    files = [f for f in os.listdir(subfolder_path) if (f.endswith('.mp4') or f.endswith('.avi'))]
    # Update the file_selector options
    file_selector.options = files

# Function to play selected video
def play_video(change):
    clear_output(wait=True)
    folder = folder_selector.value
    filename = file_selector.value
    if filename:
        video_path = os.path.join(base_path, folder, filename)
        video_clip = VideoFileClip(video_path)
        video_resized = video_clip.resize(newsize=(512, 512))
        # Display the video
        display(folder_selector)  # re-display the folder selector
        display(file_selector)    # re-display the file selector
        display(play_button)      # re-display the play button
        display(video_resized.ipython_display())
    else:
        print("Please select a video")

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

# Create a dropdown menu for the folders
folder_selector = widgets.Dropdown(
    options=subfolders,
    description='Select Folder:',
    disabled=False,
)

# Set up the file selector without any options for now
file_selector = widgets.Dropdown(
    description='Select Video:',
    disabled=False,
)

# Initially populate file_selector with the first subfolder's content
update_file_selector()

# Link the folder selector's change event to update_file_selector function
folder_selector.observe(update_file_selector, 'value')

# Create a button to play the video
play_button = widgets.Button(description="Play Video")
play_button.on_click(play_video)

# Display the dropdown menus and play button
display(folder_selector)
display(file_selector)
display(play_button)


Dropdown(description='Select Folder:', index=7, options=('Usetablet2', 'Drink', 'WatchTV', 'Usecomputer', 'Eat…

Dropdown(description='Select Video:', index=1, options=('Test.mp4', 'dance.mp4'), value='dance.mp4')

Button(description='Play Video', style=ButtonStyle())

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



t: 100%|██████████| 47/47 [00:01<00:00, 37.44it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/ict3104-team01-2023/data/videos/youtube/dance.mp4, 2764800 bytes wanted but 0 bytes read,at frame 46/47, at time 1.53/1.55 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

                                                            

Moviepy - Done !
Moviepy - video ready __temp__.mp4


## **Inference Section** 💡💡💡
####This section allows you to perform inference using a pretrained genAI model based on the Charades project.  
-------------------
  
####How to use:
1. Create the configs for a model by adding in the prompts and name
2. Run the generate videos cell to view output videos with captions with a DEFAULT skeleton file
3. Video will be saved in a new folder called GeneratedVideos
4. Refresh the folder if the ouptut folder is not shown after the video genration cell is done loading

### Customisation of configs (aka models)

In [6]:
pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.7 MB/s eta 0:00:00


In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import shutil
from collections import OrderedDict
import ruamel.yaml

yaml = ruamel.yaml.YAML()
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

config_template = '/content/ict3104-team01-2023/FollowYourPoseTeam1/configs/pose_sample_template.yaml'

# Define a custom layout for the input widget
input_layout = widgets.Layout(width='500px')

button_layout = widgets.Layout(width='250px')

# Create input widgets with the custom layout
file_name_input = widgets.Text(
    value='',
    placeholder='Config name here (no .yaml in string)',
    description='Config name:',
    disabled=False,
    layout=input_layout
)

video_length_input = widgets.IntText(
    value=8,
    description='Video Length:',
    disabled=False,
    layout=input_layout
)

width_input = widgets.IntText(
    value=512,
    description='Width:',
    disabled=False,
    layout=input_layout
)

height_input = widgets.IntText(
    value=512,
    description='Height:',
    disabled=False,
    layout=input_layout
)

# Container to hold dynamically created prompt input fields
prompt_inputs = [
    widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description='Prompt 1:',
        disabled=False,
        layout=input_layout
    )
]

# Function to handle the click of "Add Prompt" button
def add_prompt(button):
    # Create a new prompt input field
    new_prompt = widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description=f'Prompt {len(prompt_inputs) + 1}:',
        disabled=False,
        layout=input_layout
    )
    prompt_inputs.append(new_prompt)

    # Redraw all UI elements
    redraw_ui()

# Create an "Add prompt" button
add_prompt_button = widgets.Button(description="Add Prompt", layout=button_layout)
add_prompt_button.on_click(add_prompt)

def remove_latest_prompt(button):
    if len(prompt_inputs) > 1:  # To ensure there's always at least one prompt
        prompt_inputs.pop()
        redraw_ui()

remove_prompt_button = widgets.Button(description="Remove Latest Prompt", layout=button_layout)
remove_prompt_button.on_click(remove_latest_prompt)

def copy_file(button):
    # Check for config name and at least one valid prompt
    if not file_name_input.value or not any(prompt.value for prompt in prompt_inputs):
        clear_output(wait=True)
        redraw_ui()
        print("Please specify a config name and at least one prompt.")
        return

    # Redraw UI
    redraw_ui()

    # Check if the template file exists
    if os.path.exists(config_template):
        with open(config_template, 'r') as f:
            content = yaml.load(f)

        # Update the yaml content with input values
        content['validation_data']['video_length'] = video_length_input.value
        content['validation_data']['width'] = width_input.value
        content['validation_data']['height'] = height_input.value
        content['validation_data']['prompts'] = [prompt.value for prompt in prompt_inputs if prompt.value]

        # Get the user's input for the file name
        dest_file_name = file_name_input.value + ".yaml"

        # Define the destination path for the copied file
        dest_path = '/content/ict3104-team01-2023/data/pre_trained_models/' + dest_file_name

        # Write the updated content to the new file
        with open(dest_path, 'w') as f:
            yaml.dump(content, f)

        print(f"YAML file {dest_file_name} generated at: {dest_path}")
    else:
        print("Template file not found!")


copy_button = widgets.Button(description="Generate Config", layout=button_layout)
copy_button.on_click(copy_file)

def redraw_ui():
    clear_output(wait=True)
    display(file_name_input, video_length_input, width_input, height_input)
    for prompt in prompt_inputs:
        display(prompt)
    display(add_prompt_button, remove_prompt_button, copy_button)

# Display the initial UI components
redraw_ui()


Text(value='test1', description='Config name:', layout=Layout(width='500px'), placeholder='Config name here (n…

IntText(value=8, description='Video Length:', layout=Layout(width='500px'))

IntText(value=512, description='Width:', layout=Layout(width='500px'))

IntText(value=512, description='Height:', layout=Layout(width='500px'))

Text(value='Batman in the car', description='Prompt 1:', layout=Layout(width='500px'), placeholder='Type your …

Button(description='Add Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Remove Latest Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Generate Config', layout=Layout(width='250px'), style=ButtonStyle())

YAML file test1.yaml generated at: /content/ict3104-team01-2023/data/pre_trained_models/test1.yaml


In [8]:
import ipywidgets as widgets
from IPython.display import display
import os

# Assuming your folder path is '/content/ict3104-team01-2023/data/pre_trained_models/'
folder_path = '/content/ict3104-team01-2023/data/pre_trained_models/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create an empty dropdown menu
file_selector = widgets.Dropdown(
    options=[],
    description='Select Model:',
    disabled=False,
)

# Display the dropdown menu
display(file_selector)

# Update the dropdown options with the file list
file_selector.options = files


Dropdown(description='Select Model:', options=(), value=None)

In [9]:
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output

%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
print("Selected "+file_selector.value)

def run_command(change):
  config_file = file_selector.value
  modelOriginPath = "/content/ict3104-team01-2023/data/pre_trained_models/"
  model = modelOriginPath + config_file
  print("Working on path" + model)
  # Run the command and capture the output
  process = subprocess.Popen(f'TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="{model}"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

  # Display the output in real-time
  for line in process.stdout:
      print(line, end="")  # Display the output line by line

  process.wait()  # Wait for the command to finish
generate_video_button = widgets.Button(description="Generate Video")
generate_video_button.on_click(run_command)

#command = f'TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="{model}"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"'

# Display the button
display(generate_video_button)

/content/ict3104-team01-2023/FollowYourPoseTeam1
Selected default.yaml


Button(description='Generate Video', style=ButtonStyle())

##**Training Section** 🏋🏼‍♂️🏋🏼‍♂️🏋🏼‍♂️
####This section allows you to train a new genAI model based on the Charades project.
<br> NOT IMPLEMENTED
-------------------

## Generate Skeletons

In [10]:
!pip install openmim
!pip install tqdm
!mim install mmengine==0.8.5
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.0.0"
%cd /content
%mkdir /content/mmpose/vis_results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.0/443.0 kB 26.2 MB/s eta 0:00:00
  Preparing metadat

Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.5/437.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 31.4 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 MB 9.6 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
/content


In [11]:
import os
os.chdir('/content/mmpose')
!pip install -e .
import mmpose

Obtaining file:///content/mmpose
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-m26jgy28/xtcocotools_db885f3975c54f70a2586b939bf66cab
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-m26jgy28/xtcocotools_db885f3975c54f70a2586b939bf66cab
  Resolved https://github.com/jin-s13/xtcocoapi to commit d74033ff1635e9002133b2380862bc2b728584d2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58286 sha256=61e5402ec2799d3d8b10a578d5b49ee4a75f801dd2e0be6b17ed764ad5a521b9
  Stored in directory: /root/.cache/pip/wheels/31/c3/b2/b4ada5f44179429aa1a5a86ea2d3dba9aaa323703a08b0aa51
  Created wheel for xtcocotools: filename=xtcocotools-1.14.3-cp310-cp310-linux_x86_64.whl size=379765 sha256=735f5827b5de25d7

###Choose a video and click Generate to start generating the skeleton.

<b>Note: only .mp4 and .avi videos are supported.</b>
<b>To train multiple videos at once, click the checkbox, and choose the folder in which multiple videos reside.</b>

- Blue cube means that the training is still in progress  
- Green cube means that the training is completed  
- The skeleton can be found in mmpose/vis_results
- Ensure that the video file names do not contain "-“ or space in it

In [16]:
# @title
# #restart runtime before running this code for import MMPose to work
import os
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
from moviepy.editor import *
import subprocess

from tqdm.notebook import tqdm

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]

# Collect all .mp4 files from the subfolders with headers
files = []
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and f.endswith('.mp4'):
            files.append(f"{subfolder} - {f}")

# Create a checkbox...
multiple_videos = widgets.Checkbox(
    value=False,
    description='Generate Multiple Videos?',
    disabled=False,
    indent=False
)

# Create a dropdown menu with the folder names
folder_selector = widgets.Dropdown(
    options=subfolders,
    description='Select Folder:',
    disabled=False,
)

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Video:',
    disabled=False,
)

visuals_dir = '/content/mmpose/vis_results'
os.makedirs(visuals_dir, exist_ok=True)

# Define a function to update the file_selector options
def update_file_options(change):
    selected_folder = folder_selector.value
    folder_path = os.path.join(base_path, selected_folder)
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    video_files = [f"{selected_folder} - {f}" for f in files if f.endswith('.mp4') or f.endswith('.avi')]
    file_selector.options = video_files


def generate(folder, filename):
    print('Generating skeleton, please hold on')
    print(folder, filename)
    img_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
    main_file_path = '/content/mmpose'
    new_file_name = os.path.splitext(filename)[0]
    output_path = os.path.join(main_file_path, 'vis_results', new_file_name)

    inference_path = os.path.join(main_file_path, 'demo', 'inferencer_demo.py')

    shell_command = f"python {inference_path} {img_path} --pose2d human --black-background --thickness 5 --vis-out-dir {output_path}"

    # Execute the shell command to generate the skeleton
    with subprocess.Popen(shell_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as proc:
        for line in iter(proc.stdout.readline, ''):
            print("stdout: " + line.strip())

        for line in iter(proc.stderr.readline, ''):
            print("stderr: " + line.strip())

    print("Skeleton generation process done.")

    # Initialize tqdm in manual mode
    pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)

    # Iterate over the results (you need to provide the appropriate result generator here)
    results = []
    print(f"Skeleton generation for {filename} done.")

    # Update the progress bar
    pbar.update(1)

    print(f"{filename} selected.")


def generate_skeleton(change):
    clear_output(wait=True)
    if (multiple_videos.value):
        folder_path = os.path.join(base_path, folder_selector.value)
        completed = []
        for filename in os.listdir(folder_path):
              if filename.endswith('.mp4') or filename.endswith('.avi') :
                  folder = folder_selector.value
                  print("COMPLETED TRAININGS:")
                  print("======================")
                  for completed_file in completed:
                      print(f"{completed_file}")
                  print("======================")
                  generate(folder, filename)
                  completed.append(filename)
                  clear_output(wait=True)
        for completed_file in completed:
            print(completed_file+" Skeleton Generated.")


    else:
        folder, filename = file_selector.value.split(" - ", 1)
        generate(folder, filename)
        clear_output(wait=True)
        print(filename+" Skeleton Generated in " + visuals_dir)

    #Redisplay UI
    display(multiple_videos)
    display(folder_selector)
    display(file_selector)
    display(gen_button)

# Attach the callback function to the folder_selector
folder_selector.observe(update_file_options, 'value')
clear_output(wait=True)
# Create a button to play the video
gen_button = widgets.Button(description="Generate MMPose Skeleton")
# Attach the play_video function to the button's on_click event
gen_button.on_click(generate_skeleton)

# Display the dropdown menu and play button
display(multiple_videos)
display(folder_selector)
display(file_selector)
display(gen_button)

dance.mp4 Skeleton Generated in /content/mmpose/vis_results


Checkbox(value=False, description='Generate Multiple Videos?', indent=False)

Dropdown(description='Select Folder:', index=7, options=('Usetablet2', 'Drink', 'WatchTV', 'Usecomputer', 'Eat…

Dropdown(description='Select Video:', index=1, options=('youtube - Test.mp4', 'youtube - dance.mp4'), value='y…

Button(description='Generate MMPose Skeleton', style=ButtonStyle())

##**Testing Section** 📝📝📝
####This section allows you to evaluate a trained model based on the Charades project.
-------------------
  
####How to use:
1. Load the models generated from the inference section
2. Select up to 5 models to generate the output videos
3. Select a skeleton for the output video
4. Run them in the next cell to obtain the output videos
5. View the videos side by side

###Load up to 5 pretrained models

In [17]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

folder_path = '/content/ict3104-team01-2023/data/pre_trained_models/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a layout for the train_file_selector with adjusted width
layout = widgets.Layout(width='350px')  # Adjust width as needed

# Create a multiple-selection widget with the new layout
train_file_selector = widgets.SelectMultiple(
    options=files,
    disabled=False,
    rows=5,  # Number of visible rows
    layout=layout
)

# Define the directory containing subfolders with skeleton videos
skeleton_directory = "/content/mmpose/vis_results"

# List subfolders in the specified directory
skeleton_subfolders = [folder for folder in os.listdir(skeleton_directory) if os.path.isdir(os.path.join(skeleton_directory, folder))]


# Function to list .avi files in a directory
def list_skeletons(directory):
    skeletons = [f for f in os.listdir(directory) if f.endswith(".avi") or f.endswith(".mp4") or f.endswith(".mov")]
    return skeletons

# Access .avi files inside the subfolders
video_files_in_subfolders = {}
for subfolder in skeleton_subfolders:
    subfolder_path = os.path.join(skeleton_directory, subfolder)
    skeletons = list_skeletons(subfolder_path)
    video_files_in_subfolders[subfolder] = skeletons


# Create a dropdown widget for selecting .avi files
skeleton_selector = widgets.Dropdown(
    options={subfolder: list_skeletons(os.path.join(skeleton_directory, subfolder)) for subfolder in skeleton_subfolders},
    description='Select Skeleton'
)


# Create a layout for the combined widget with adjusted width
layout = widgets.Layout(width='350px')  # Adjust width as needed

def on_values_change(change):
    if len(train_file_selector.value) > 5:
        # Display a warning message and deselect the last-selected value
        clear_output(wait=True)
        print("You can select up to 5 files only!")
        display(instruction)
        display(train_file_selector)
        # Deselect the last selected option
        train_file_selector.value = train_file_selector.value[:-1]

# Attach the callback function to the value trait of the train_file_selector
train_file_selector.observe(on_values_change, 'value')

# Create an instruction widget using HTML
instruction = widgets.HTML(
    value="<i>Use CTRL or SHIFT to select multiple files.</i><br><i>Select Models:</i>",
    placeholder='',
    description='',
)

# Create an output widget for displaying results
output = widgets.Output()

# Create a button for passing the selection to the next cell
pass_selection_button = widgets.Button(description="Pass Selection to Next Cell")
pass_selection_button.layout = layout

def pass_selection_to_next_cell(change):
    # Output the selected value
    print("Selected Skeleton:", skeleton_selector.value)
    print("Selected models: ", train_file_selector.value)

# Attach the callback function to the button
pass_selection_button.on_click(pass_selection_to_next_cell)

# Create a widget that contains both the model selection and skeleton subfolder selection
combined_widget = widgets.VBox([
    widgets.HTML(value="<i>Use CTRL or SHIFT to select multiple files.</i><br><i>Select Models:</i>"),
    train_file_selector,
    skeleton_selector,
    pass_selection_button
])

# Display the combined widget
display(combined_widget)

# Display the output widget
display(output)

Output()

Selected Skeleton: ['dance.mp4']
Selected models:  ('test1.yaml',)


### Load the models to generate the videos with caption
1. Wait for the cell to run, it will take some time
2. Track the progress from the loading bar and output of the cell
3. If the error message "CalledProcessError: Command .... " shows up, the GPU resource of colab has been used up, wait for a day to reuse the notebook

In [18]:
import subprocess
import os

print(train_file_selector.value)
print(skeleton_selector.value[0])
skeleton_selector_file = skeleton_selector.value[0]

%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
modelOriginPath = "/content/ict3104-team01-2023/data/pre_trained_models/"
skeletonOriginPath = f"/content/mmpose/vis_results/{skeleton_selector_file[:-4]}"
print(f"{skeletonOriginPath}/{skeleton_selector_file}")

for config_file in train_file_selector.value:
    # Set the command with the current config file
    print("Working on: " + config_file)
    config_file = modelOriginPath + config_file
    command = f'TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="{config_file}"  --skeleton_path="{skeletonOriginPath}/{skeleton_selector_file}"'

    # Run the command and capture the output
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    # Display the output in real-time
    for line in process.stdout:
        print(line, end="")  # Display the output line by line

    process.wait()  # Wait for the command to finish

('test1.yaml',)
dance.mp4
/content/ict3104-team01-2023/FollowYourPoseTeam1
/content/mmpose/vis_results/dance/dance.mp4
Working on: test1.yaml
2023-11-20 05:02:39.088015: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 05:02:39.088071: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 05:02:39.088105: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 05:02:39.096621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructi

###Play videos side by side

In [19]:
import ipywidgets as widgets
from IPython.display import display, HTML, Video
from moviepy.editor import *
from IPython.display import clear_output
from base64 import b64encode
from ipywidgets import Output, GridspecLayout
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import glob


def play_video(change):
    clear_output(wait=True)  # clear the current cell output before displaying new video
    if file_selector1.value and file_selector2.value:

            originPath = '/content/GeneratedVideos'
            selected_folder = folder_selector.value  # Get the selected folder name

            video_path1 = os.path.join(originPath, selected_folder, file_selector1.value)
            video_path2 = os.path.join(poses_base_path, file_selector2.value[:-4], file_selector2.value)
            video_clip1 = VideoFileClip(video_path1)
            video_clip2 = VideoFileClip(video_path2)

            # Resize the videos to the desired dimensions
            video_resized1 = video_clip1.resize(newsize=(512, 512))
            video_resized2 = video_clip2.resize(newsize=(512, 512))

            # You'll need to save the resized video temporarily
            temp_gif = '/tmp/resized_video1.mp4'
            temp_skeleton = '/tmp/resized_video2.mp4'
            temp_edit = '/tmp/edited.mp4'

            # Write the resized video clips to the temporary files
            video_resized1.write_videofile(temp_gif)
            video_resized2.write_videofile(temp_skeleton)

            # Edit video based on inputs
            ffmpeg_extract_subclip(temp_skeleton,0,0.544,targetname=temp_edit)
            !ffmpeg -y -i {temp_edit} -vf "setpts=1.65*PTS" {temp_skeleton}


            # Superimpose the clips
            gif_result_clip = VideoFileClip(temp_gif, target_resolution=(1080,1920))
            skeleton_result_clip = VideoFileClip(temp_skeleton, target_resolution=(1080,1920))
            masked_clip = vfx.mask_color(skeleton_result_clip,color=[0,0,0])
            superimposed_clip = CompositeVideoClip([gif_result_clip,masked_clip])
            superimposed_clip = superimposed_clip.resize(newsize=(512, 512))
            superimposed_clip.write_videofile(temp_edit)

            clear_output(wait=True)

            # Create a GridspecLayout for displaying the videos side by side
            grid = GridspecLayout(1, 3)

            container = widgets.VBox([grid], layout = widgets.Layout(
                border='3px solid black',
                justify_content = 'center',
                align_items='center'
            ))

            out1 = Output()
            out2 = Output()
            out3 = Output()

            with out1:
                display(Video(temp_gif, embed=True,html_attributes="loop autoplay"))
            with out2:
                display(Video(temp_skeleton, embed=True,html_attributes="loop autoplay"))
            with out3:
                display(Video(temp_edit, embed=True,html_attributes="loop autoplay"))

            grid[0, 0] = out1
            grid[0, 1] = out2
            grid[0, 2] = out3

            display(container)

            # Display the videos
            display(folder_selector)
            display(update_files_button)
            display(file_selector1)  # re-display the first dropdown
            display(file_selector2)  # re-display the second dropdown
            display(play_button)  # re-display the play button
    else:
        print("Please select two videos")

# Base path
base_path = '/content/GeneratedVideos'
poses_base_path = '/content/mmpose/vis_results'

folder_names = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

files = []
# Use glob to get files with extension .mp4
mp4_files = glob.glob(os.path.join(poses_base_path, '**/*.mp4'), recursive=True)

# Use glob to get files with extension .mov
mov_files = glob.glob(os.path.join(poses_base_path, '**/*.mov'), recursive=True)

# Use glob to get files with extension .avi
avi_files = glob.glob(os.path.join(poses_base_path, '**/*.avi'), recursive=True)

# Concatenate the lists of files
skeleton_files = mp4_files + mov_files + avi_files

# Create a folder selection dropdown
folder_selector = widgets.Dropdown(
    options=folder_names,
    description='Select Folder:',
    disabled=False,
)

# Create a button to update the file options when a folder is selected
update_files_button = widgets.Button(description="Select Folder")

def update_files_button_clicked(b):
    global selected_folder
    selected_folder = os.path.join(base_path, folder_selector.value)
    files = glob.glob(os.path.join(selected_folder, '**/*.mp4'), recursive=True)
    # Extract only the file names without the path
    file_names = [os.path.basename(file) for file in files]
    file_selector1.options = file_names
    skeleton_file_names = [os.path.basename(file) for file in skeleton_files]
    file_selector2.options = skeleton_file_names

update_files_button.on_click(update_files_button_clicked)

# Create two dropdown menus with the file names
file_selector1 = widgets.Dropdown(
    options=files,
    description='Select Origin Video:',
    disabled=False,
)

file_selector2 = widgets.Dropdown(
    options=skeleton_files,
    description='Select Skeleton Video:',
    disabled=False,
)

# Create a button to play the videos
play_button = widgets.Button(description="Select Videos")

# Attach the play_video function to the button's on_click event
play_button.on_click(play_video)

# Display the folder selection dropdown and update button
display(folder_selector)
display(update_files_button)

# Initialize selected_folder
selected_folder = os.path.join(base_path, folder_selector.value)
update_files_button_clicked(None)  # Update file options for the initial folder

# Display the video selection dropdowns and play button
display(file_selector1)
display(file_selector2)
display(play_button)

Dropdown(description='Select Folder:', options=('test1-2023-11-20 05:03:57.665268',), value='test1-2023-11-20 …

Button(description='Select Folder', style=ButtonStyle())

Dropdown(description='Select Origin Video:', index=1, options=('Batman in the car.mp4', 'Batman in the car_wit…

Dropdown(description='Select Skeleton Video:', options=('dance.mp4',), value='dance.mp4')

Button(description='Select Videos', style=ButtonStyle())